In [ ]:
#| default_exp core

# fhdaisy.core
> Builder API used behind the scenes to create the basic components

In [ ]:
#| export
from fastcore.utils import *
from fasthtml.common import *
import fasthtml.components as fh
from fasthtml.jupyter import *

import inspect

In [ ]:
from fastcore.test import *

## The basics

In [ ]:
#| export
daisy_link = Link(href='https://cdn.jsdelivr.net/npm/daisyui@5', rel='stylesheet', type='text/css')
tw_scr = Script(src='https://cdn.jsdelivr.net/npm/@tailwindcss/browser@4')
daisy_hdrs = (daisy_link, tw_scr)

In [ ]:
#| export
def mk_previewer(app=None, cls=f'max-w-lg'):
    xcls = cls
    if not app: app=FastHTML(hdrs=daisy_hdrs)
    def p(*c, cls='', **kw):
        return HTMX(Div(cls=f'{xcls} {cls}')(*c), app=app, host=None, port=None, **kw)
    return p

In [ ]:
p = mk_previewer()

In [ ]:
c = Button('Hey there', cls='btn')
p(c)

In [ ]:
print(c)

<button class="btn">Hey there</button>


## Creating simple components

In [ ]:
#| export
def hyphens2camel(x): return ''.join(o.title() for o in x.split('-'))
hyphens2camel('chat-bubble')

'ChatBubble'

Tailwind utility classes can start with `-` (like `-mt-5` or `-right-5`). We need to distinguish these from our shorthand where we prepend the class name to user provided values that start with `-` (e.g. `Tooltip('-left')` becomes `tooltip-left`)

In [ ]:
#| export
_neg_twu_pfxs = set('mt ml mr mb mx my translate rotate scale skew inset top bottom left right z space'.split())
def _is_neg_twu(x): return x[0]=='-' and len(parts:=x[1:].split('-'))>=2 and parts[0] in _neg_twu_pfxs

In [ ]:
for o in ('-mt-5', '-right-5'): test_eq(_is_neg_twu(o), True)
for o in ('-right', 'mt-5', 'right-5'): test_eq(_is_neg_twu(o), False)

In [ ]:
#| export
def mk_compfn(compcls, tag=None, name=None, xcls='', **compkw):
    if not name: name=hyphens2camel(compcls)
    if not tag: tag=name
    compfunc = getattr(fh, tag)

    def fn(*c, cls='', **kw):
        cls = ' '.join(f'{compcls if x[0]=="-" and not _is_neg_twu(x) else ""}{x}' for x in cls.split())
        return compfunc(*c, cls=f'{compcls} {cls} {xcls}', **compkw, **kw)

    fn.__name__ = name
    inspect.currentframe().f_back.f_globals[name] = fn

In [ ]:
mk_compfn('btn', 'Button')

In [ ]:
c = Btn('Hey there', cls='-primary p-5 text-2xl rounded-full')
print(c)

<button class="btn btn-primary p-5 text-2xl rounded-full ">Hey there</button>


In [ ]:
p(c)

Demonstrating that negative utility classes are handled correctly:

In [ ]:
mk_compfn('tooltip', 'Div')

In [ ]:
c = Tooltip('Main text is offset to the right', cls='-right-50 -primary -left', data_tip='Tooltip appears to the left')
print(c)

<div data-tip="Tooltip appears to the left" class="tooltip -right-50 tooltip-primary tooltip-left ">Main text is offset to the right</div>


In [ ]:
p(c)